# DialEval-1 DNN practice (with evaluation function)
### Model : CNN, LSTM
##### input round, sender pred label
##### Edited by Weber Huang in 2020-05-28

### 1. Eval function and customizes loss function

In [1]:
from scipy import stats
import tensorflow as tf

def normalize(pred, truth):
    """ convert inputs to np.array and make sure
    inputs are normalized probability distributions
    """
    if len(pred) != len(truth):
        raise ValueError("pred and truth have different lengths")
    if len(pred) == 0 or len(truth) == 0:
        raise ValueError("pred or truth are empty")

    pred, truth = np.asarray(pred), np.asarray(truth)
    if not ((pred >= 0).all() and (truth >= 0).all()):
        raise ValueError("probability distribution should not be negative")
    pred, truth = pred / pred.sum(), truth / truth.sum()
    return pred, truth

def jensen_shannon_div(pred, truth, base=2):
    ''' JSD: Jensen-Shannon Divergence
    '''
    pred, truth = normalize(pred, truth)
    m = 1. / 2 * (pred + truth)
    return (stats.entropy(pred, m, base=base)
            + stats.entropy(truth, m, base=base)) / 2.

def root_normalized_squared_error(pred, truth):
    """ RNSS: Root Normalised Sum of Squares
    """

    def squared_error(pred, truth):
        return ((pred - truth) ** 2).sum()

    pred, truth = normalize(pred, truth)
    return np.sqrt(squared_error(pred, truth) / 2)

def jsd_custom_loss(y_true, y_pred):
            
    # calculate loss, using y_pred
    ''' JSD: Jensen-Shannon Divergence
    '''
#     y_pred, y_true = normalize(y_pred, y_true)
    m = 1. / 2 * (y_pred + y_true)
    # loss = (stats.entropy(y_pred, m, base=2) + stats.entropy(y_true, m, base=2)) / 2.
    # tf.keras.losses.KLD()
    loss = (tf.keras.losses.KLD(y_pred, m) + tf.keras.losses.KLD(y_true, m)) / 2.
    return loss
  
# model.compile(loss=jsd_custom_loss, optimizer='adam')

def rnss_custom_loss(y_true, y_pred):
            
    # calculate loss, using y_pred
    """ RNSS: Root Normalised Sum of Squares
    """

    def squared_error(y_pred, y_true):
        return ((y_pred - y_true) ** 2).sum()

#     y_pred, y_true = normalize(y_pred, y_true)
    loss = np.sqrt(squared_error(y_pred, y_true) / 2)
    
    return loss
  
# model.compile(loss=custom_loss, optimizer='adam')

C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

### 2. input dataset and modify

In [2]:
import os
import pandas as pd

os.chdir('C:/Users/doudi/OneDrive/Documents/ntcir15/Dataset/DialEval-1')
df = pd.read_excel(r'./200514_dev+train.xlsx')

In [17]:
tmp = []
for i in df['sender']:
    if i == 'customer':
        tmp.append(0)
    else:
        tmp.append(1)
df['sender_num'] = tmp
df.head(20)

,id,round,sender,texts,max_label,round_label,CNUG,CNUG*,CNUG0,CNaN,HNUG,HNUG*,HNaN,sender_num
0,4227729258237820,1,customer,内涵 段子 联通 皮 点赞 中国联通 中国联通 客服 掌上 营业厅 内涵 段子 话题 封 郑...,3,2,0.052632,0.000000,0.157895,0.789474,0.000000,0.000000,0.000000,0
1,4227729258237820,2,helpdesk,u,6,4,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.842105,1
2,4227729258237820,3,customer,夸夸,3,0,0.157895,0.000000,0.000000,0.842105,0.000000,0.000000,0.000000,0
3,4227729258237820,4,helpdesk,*,6,4,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.842105,1
4,4121001149457180,1,customer,距离 反映 问题 已经 一个 星期 花粉 助手 D 荣耀 honor 荣耀 手机 华为 终端...,2,2,0.052632,0.000000,0.789474,0.157895,0.000000,0.000000,0.000000,0
5,4121001149457180,2,helpdesk,您好 建议您 先 确认 手机 系统 版本 是否 最新 提示 升级 建议 备份 升 最新 版本...,4,4,0.000000,0.000000,0.000000,0.000000,0.526316,0.105263,0.368421,1
6,4121001149457180,3,customer,遗憾 告诉您 网店 表示 解决不了 皮球 踢,0,0,0.631579,0.000000,0.052632,0.315789,0.000000,0.000000,0.000000,0
7,4121001149457180,4,helpdesk,亲 去过 售后,4,4,0.000000,0.000000,0.000000,0.000000,0.684211,0.000000,0.315789,1
8,4121001149457180,5,customer,*,0,0,0.684211,0.000000,0.000000,0.315789,0.000000,0.000000,0.000000,0
9,4121001149457180,6,helpdesk,您好 心情 D 非常 理解 请 放心 D 会 尽我所能 帮 解决问题 烦请 私信 留下 联系...,4,4,0.000000,0.000000,0.000000,0.000000,0.578947,0.000000,0.421053,1


In [10]:
# c_df = df[['id','round','sender','texts','max_label','round_label','CNUG','CNUG*','CNUG0','CNaN']]
# h_df = df[['id','round','sender','texts','max_label','round_label','HNUG','HNUG*','HNaN']]

In [12]:
# h_df.shape

(17155, 9)

In [5]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras import optimizers
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import Dense, Embedding
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import concatenate
from keras import initializers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM



Using TensorFlow backend.


In [13]:
dev = df[0:1754]
train = df[1755:]

X_train = train.filter(['round','sender_num','round_label','texts'])
X_test = dev.filter(['round','sender_num','round_label','texts'])
y_train_c = train.filter(['CNUG','CNUG*','CNUG0','CNaN'])
y_test_c = dev.filter(['CNUG','CNUG*','CNUG0','CNaN'])
y_train_h = train.filter(['HNUG','HNUG*','HNaN'])
y_test_h = dev.filter(['HNUG','HNUG*','HNaN'])
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [14]:
X1_train = X_train['texts']
#     X1_train = X_train[:,[-1]]
X1_train = [str (item) for item in X1_train]
    
X1_test = X_test['texts']
#     X1_test = X_test[:,[-1]]
X1_test = [str (item) for item in X1_test]

X2_train = X_train[['round','sender_num','round_label']].values
#     X2_train = X_train[:,0:7]
X2_test = X_test[['round','sender_num','round_label']].values
#     X2_test = X_test[:,0:7]
    
token = Tokenizer(num_words = 20000)
token.fit_on_texts(X1_train)
vocab = token.word_index
print(token.document_count)

x_train_seq = token.texts_to_sequences(X1_train)
x_test_seq = token.texts_to_sequences(X1_test)
X1_train = sequence.pad_sequences(x_train_seq, maxlen = 150)
X1_test = sequence.pad_sequences(x_test_seq, maxlen = 150)

#     y_one_train = np_utils.to_categorical(y_train)
#     y_one_test = np_utils.to_categorical(y_test)

15400


### 3. DNN 

In [15]:
def CNN(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
    
    num_labels = 7
    main_input = Input(shape=(150,), dtype='float64')

    sub_input = Input(shape=(3,))
    
    # pre-train embeddings
    # embedder = Embedding(len(vocab) + 1, 300, input_length = 20, weights = [embedding_matrix], trainable = False)
    # embed = embedder(main_input)
    embed = Embedding(len(vocab)+1, 300, input_length=150)(main_input)
    # filter size, region size
    cnn = Convolution1D(2, 2, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    # main_output = Dense(num_labels, activation='sigmoid')(drop)


    dense_1 = Dense(units=256,activation='relu')(sub_input)
    drop_1 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(drop_1)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([drop, dense_2])
    dense_3 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=7, activation='softmax')(dense_3)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''

def lstm(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
     
    main_input = Input(shape=(150,), dtype='float64')
    sub_input = Input(shape=(3,))
    
    embed = Embedding(output_dim=32,input_dim=20000,input_length=150)(main_input)
    dropout_1 = Dropout(0.35)(embed)
    lst = LSTM(units=16)(dropout_1)
    dense_1 = Dense(units=256,activation='relu')(lst)
    dropout_2 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(dropout_2)
    dense_3 = Dense(units=7,activation='softmax')(dense_2)


    dense_4 = Dense(units=256,activation='relu')(sub_input)
    dropout_3 = Dropout(0.35)(dense_4)
    dense_5 = Dense(units=128,activation='relu')(dropout_3)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([dense_3, dense_5])
    dense_6 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=7, activation='softmax')(dense_6)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''

def CNN_C(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
    
    num_labels = 4
    main_input = Input(shape=(150,), dtype='float64')

    sub_input = Input(shape=(3,))
    
    # pre-train embeddings
    # embedder = Embedding(len(vocab) + 1, 300, input_length = 20, weights = [embedding_matrix], trainable = False)
    # embed = embedder(main_input)
    embed = Embedding(len(vocab)+1, 300, input_length=150)(main_input)
    # filter size, region size
    cnn = Convolution1D(2, 2, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    # main_output = Dense(num_labels, activation='sigmoid')(drop)


    dense_1 = Dense(units=256,activation='relu')(sub_input)
    drop_1 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(drop_1)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([drop, dense_2])
    dense_3 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=4, activation='softmax')(dense_3)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''

def lstm_C(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
     
    main_input = Input(shape=(150,), dtype='float64')
    sub_input = Input(shape=(3,))
    
    embed = Embedding(output_dim=32,input_dim=20000,input_length=150)(main_input)
    dropout_1 = Dropout(0.35)(embed)
    lst = LSTM(units=16)(dropout_1)
    dense_1 = Dense(units=256,activation='relu')(lst)
    dropout_2 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(dropout_2)
    dense_3 = Dense(units=4,activation='softmax')(dense_2)


    dense_4 = Dense(units=256,activation='relu')(sub_input)
    dropout_3 = Dropout(0.35)(dense_4)
    dense_5 = Dense(units=128,activation='relu')(dropout_3)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([dense_3, dense_5])
    dense_6 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=4, activation='softmax')(dense_6)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''

    
def CNN_H(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
    
    num_labels = 3
    main_input = Input(shape=(150,), dtype='float64')

    sub_input = Input(shape=(3,))
    
    # pre-train embeddings
    # embedder = Embedding(len(vocab) + 1, 300, input_length = 20, weights = [embedding_matrix], trainable = False)
    # embed = embedder(main_input)
    embed = Embedding(len(vocab)+1, 300, input_length=150)(main_input)
    # filter size, region size
    cnn = Convolution1D(2, 2, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    # main_output = Dense(num_labels, activation='sigmoid')(drop)


    dense_1 = Dense(units=256,activation='relu')(sub_input)
    drop_1 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(drop_1)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([drop, dense_2])
    dense_3 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=3, activation='softmax')(dense_3)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''

def lstm_H(X1_train, X2_train, X1_test, X2_test, y_train, y_test, loss='categorical_crossentropy'):
     
    main_input = Input(shape=(150,), dtype='float64')
    sub_input = Input(shape=(3,))
    
    embed = Embedding(output_dim=32,input_dim=20000,input_length=150)(main_input)
    dropout_1 = Dropout(0.35)(embed)
    lst = LSTM(units=16)(dropout_1)
    dense_1 = Dense(units=256,activation='relu')(lst)
    dropout_2 = Dropout(0.35)(dense_1)
    dense_2 = Dense(units=128,activation='relu')(dropout_2)
    dense_3 = Dense(units=3,activation='softmax')(dense_2)


    dense_4 = Dense(units=256,activation='relu')(sub_input)
    dropout_3 = Dropout(0.35)(dense_4)
    dense_5 = Dense(units=128,activation='relu')(dropout_3)
    # sub_output = Dense(units=2,activation='sigmoid')(dense_2)

    merge = concatenate([dense_3, dense_5])
    dense_6 = Dense(units=10, activation='relu')(merge)
    output = Dense(units=3, activation='softmax')(dense_6)

    model = Model(inputs=[main_input, sub_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, 
                              batch_size=64, verbose=2, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return train_history, pre_probability
    '''
    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(y_test, predicted)))

    print("f1_score :\n%s\n" % ( metrics.f1_score(y_test, predicted, average='macro')))
    print("acc_score :\n%s\n" % ( metrics.accuracy_score(y_test, predicted)))
    '''
    

In [7]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
'''
n_folds = 3
model_history = []
for i in tqdm(range(n_folds)):
    print("Training on Fold: ",i+1)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                        random_state = np.random.randint(1,1000, 1)[0])
                                               
    
    model_history.append(CNN(X_train, X_test, y_train, y_test))
    print("======="*12, end="\n\n\n")
'''

'\nn_folds = 3\nmodel_history = []\nfor i in tqdm(range(n_folds)):\n    print("Training on Fold: ",i+1)\n    from sklearn.model_selection import train_test_split\n    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, \n                                                        random_state = np.random.randint(1,1000, 1)[0])\n                                               \n    \n    model_history.append(CNN(X_train, X_test, y_train, y_test))\n    print("======="*12, end="\n\n\n")\n'

In [8]:
# use categorical_crossentropy as loss function
train_history , pred = CNN(X_train, X_test, y_train, y_test)

15400

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     6533100     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 150, 2)       1202        embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3)            0                                            
_____________________________________________________________________________________

Epoch 50/50
 - 2s - loss: 0.7470 - accuracy: 0.8622 - val_loss: 0.8228 - val_accuracy: 0.8458
1754/1754 [==============================] - 0s 59us/step
Test Score: 0.8152590429361489
Test Accuracy: 0.8472064137458801


In [ ]:
# use categorical_crossentropy as loss function
train_history_ls , pred_ls = lstm(X_train, X_test, y_train, y_test)

15400
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 32)      640000      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 150, 32)      0           embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 16)           3136        dropout_3[0][0]                  
______________________________________________________________________________________

In [ ]:
# use jsd as loss function
train_history_jsd , pred_jsd = CNN(X_train, X_test, y_train, y_test, loss = jsd_custom_loss)

In [ ]:
# use jsd as loss function
train_history_ls_jsd , pred_ls_jsd = lstm(X_train, X_test, y_train, y_test, loss = jsd_custom_loss)

In [ ]:
# use rnss as loss function
# train_history_rnss , pred_rnss = CNN(X_train, X_test, y_train, y_test, loss = rnss_custom_loss)

In [ ]:
# use rnss as loss function
# train_history_ls_rnss , pred_ls_rnss = lstm(X_train, X_test, y_train, y_test, loss = rnss_custom_loss)

In [ ]:
true = np.array(y_test)
def calculate_divergence(true, pred):
    tmp = 0
    for i in range(len(true)):
        tmp = tmp + jensen_shannon_div(pred[i], true[i])
    
    print('mean jsd :', tmp/len(true))
    
    tmp = 0
    for i in range(len(true)):
        tmp = tmp + root_normalized_squared_error(pred[i], true[i])
    
    print('mean rnss :', tmp/len(true))
    
#         print('---sentence{0}---'.format(i))
#         print('jsd :', jensen_shannon_div(pred[i], true[i]))
#         print('rnss :', root_normalized_squared_error(pred[i], true[i]))

# print('--- textCNN ---', '\n')
# calculate_divergence(true, pred)

# print('--- LSTM ---', '\n')
# calculate_divergence(true, pred_ls)

In [ ]:
print('=== categorical_crossentropy ===', '\n')
print('---textCNN---')
calculate_divergence(true, pred)
print('---LSTM---')
calculate_divergence(true, pred_ls)
print('\n', '=== loss_jsd ===', '\n')
print('---textCNN---')
calculate_divergence(true, pred_jsd)
print('---LSTM---')
calculate_divergence(true, pred_ls_jsd)

In [18]:
submission_cnn_entropy = 

array([[5.3635314e-02, 2.3085409e-04, 8.4093058e-01, ..., 2.0889031e-04,
        8.8531706e-06, 2.3545331e-04],
       [2.8267033e-07, 1.1218920e-06, 5.2591265e-10, ..., 5.0565666e-01,
        1.3791247e-01, 3.5642898e-01],
       [5.5945146e-01, 1.1985750e-01, 9.3967244e-03, ..., 5.9507645e-05,
        1.2362933e-04, 3.6611807e-05],
       ...,
       [1.8303791e-06, 1.0560428e-06, 1.4783109e-08, ..., 6.1185038e-01,
        1.0529692e-01, 2.8284660e-01],
       [4.4889548e-01, 2.0985053e-01, 7.1117044e-03, ..., 2.4526957e-05,
        7.2622061e-05, 2.2786977e-05],
       [1.0568731e-06, 9.7649570e-07, 3.0893015e-09, ..., 5.1087028e-01,
        2.0788656e-01, 2.8123870e-01]], dtype=float32)

#### customer and helpdesk split

In [16]:
train_history_c , pred_c = CNN_C(X1_train, X2_train, X1_test, X2_test, y_train_c, y_test_c)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 300)     6533100     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 150, 2)       1202        embedding_2[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 3)            0                                            
____________________________________________________________________________________________

In [ ]:
train_history_h , pred_h = CNN_H(X1_train, X2_train, X1_test, X2_test, y_train_h, y_test_h)